In [33]:
novel_prompt = """너는 단문 소설을 쓰는 작가야. 주어진 단어를 포함하는 세 문단짜리 단문 소설을 작문해야 해.
아래 단어를 모두 포함하도록 단문 소설을 써 줘.
{keywords}
제목 없이 소설 내용만 써 줘. 주어진 단어를 활용하는 자연스러운 전개로 써 줘.
전지적 작가 시점으로 써 줘.
인물의 대사 한 개 이상을 포함해.
소설은 세 문단으로 이루어져야 해. 세 문단 이상 쓰지 마.
"""
science_prompt = """너는 과학적 사실을 설명하는 글을 쓰는 과학 저널리스트야. 주어진 과학 질문에 대한 답변을 세 문단으로 작성해야 해.
아래 과학 질문에 대한 답변을 써 줘.
{question}
제목이나 요약 없이 답변 내용만 써 줘. 질문과 관련없는 내용은 쓰지 마.
-다, 와 같은 종결어미를 사용하는 해라체로 작성해.
답변은 세 문단으로 이루어져야 해.
"""

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="spow12/Ko-Qwen2-7B-Instruct", device_map="auto")

Loading checkpoint shards: 100%|██████████| 4/4 [00:32<00:00,  8.14s/it]


In [34]:
import csv

results = []
with open('hw4_data/science_questions.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        instruction = science_prompt.format(question=row)
        messages = [
            {"role": "system", "content": "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요."},
            {"role": "user", "content": f"{instruction}"}
            ]

        outputs = pipe(messages, max_new_tokens=1000)
        results.append((row, outputs[0]["generated_text"]))

In [36]:
list_for_df = [(res[0], res[-1][-1]['content']) for res in results]

In [32]:
with open('hw4_results/qwen_science_results.txt', 'w', newline='') as file:
    for res in results:
        file.write(res[-1][-1]['content'])
        file.write("--------------------------------------------------\n\n")

In [37]:
import pandas as pd


# Convert the list into a DataFrame
df = pd.DataFrame(list_for_df, columns=["question", "answer"])  # You can rename "Names" to any column header

# Write to CSV file
df.to_csv("hw4_results/qwen_science_results.csv", index=False)

print("File written successfully!")

File written successfully!
